In [ ]:
#
# xarray is a multi-dimensional array tool which supports naming different 
# axes of arrays and having some arrays act as "coordinates" for others in
# a "dataset". 
#
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

In [ ]:
#
# We can load an example 3d dataset of surface air temperatures over North America, every 6 hours from Jan 2013 to
# Dec 2014, averaged over 2.5 x 2.5 degree horizontal areas
#
# an xarray contains "coordinate variables" and "data variables"
# Data variables are sized to align with one or more of the coordinate variables.
# This associates coordinates with individual items in a Data Varialble
#
# In xarray dataset below there is one Data Variable (called "air") and
# its axes align with coordinate variable "time", "lat", "lon". 
#
ds = xr.tutorial.load_dataset("air_temperature")
# Lets subtract 273.15 to put things in degress C
ds=ds-273.15
display( ds )

In [ ]:
#
# Quick look
# - like pandas (and geopandas) with xarray the different variables can be referenced by name.
#   both data variables and coordinate variables can be referenced by name. 
#
#   In this dataset the lon and lat coordinates are a regular mesh so we can use
#   matplotlib.contourf() to make a quick contour plot directly.
#
display( ds )
display( ds.air.coords  )
tn=0
plt.contourf(ds.lon,ds.lat,ds.air[tn,:,:])
ts=np.datetime_as_string(ds.time[tn].values,'h');
plt.title(r"Air Temperature ($^{\rm o}$C) t=%s"%(ts));plt.xlabel(r'Longitude ($^{\rm o}$E)');plt.ylabel(r'Latitude ($^{\rm o}$N)');plt.colorbar()

In [ ]:
#
# the xarray class provides multiple helper functions that encapsulate common operations
# - for example there is a helper function sel() that selects points that are close
#   to a particular set of coordinate variables. 
#
# Plot versus time near Boston (42N, 289E)
#  use ds.sel to select nearest point
#  subtract 273.15 to give Celcius
#  ( *9/5 + 32 would be Farenheit! )
#  
#

In [ ]:
blon=289
blat=42
data_near_boston=ds.sel(method="nearest",lat=[blat],lon=[blon]);
temp_near_boston=data_near_boston["air"][:,0,0];
times=data_near_boston["time"]

plt.plot( times, temp_near_boston );
plt.xticks(rotation = 90)
plt.xlabel("Date")
plt.ylabel(r"Degrees $^{\circ}$C")
plt.title("Temperature %3.1fN, %3.1fE"%(data_near_boston.lat.values[0],data_near_boston.lon.values[0]));

In [ ]:
#
#   we can use built in xarray function to get quick plots for statistics across all the points for all the times
#   
#
ds_std=ds.std(dim="time")
ds_mean=ds.mean(dim="time")
ds_max=ds.max(dim="time")
ds_min=ds.min(dim="time")
ds_diff=ds_max-ds_min
def quick_plot(d,t):
    plt.contourf(d.lon,d.lat,d.air)
    plt.xlabel(r"Longitude, $^\circ$E")
    plt.ylabel(r"Latitude, $^\circ$N")
    plt.title(t)
    plt.colorbar()
    plt.show()
    
quick_plot(ds_std,r"Temperature std. dev 2013-2014, $^\circ$C")
quick_plot(ds_mean,r"Temperature mean 2013-2014, $^\circ$C")
quick_plot(ds_max,r"Temperature max 2013-2014, $^\circ$C")
quick_plot(ds_min,r"Temperature min 2013-2014, $^\circ$C")
quick_plot(ds_diff,r"$\Delta$Temperature min 2013-2014, $^\circ$C")

In [ ]:
tmax_bos=( ds.resample(time="1MS").max() ).sel(method="nearest",lat=[blat],lon=[blon])["air"][:,0,0] 
tmin_bos=( ds.resample(time="1MS").min() ).sel(method="nearest",lat=[blat],lon=[blon])["air"][:,0,0] 
times=tmin_bos["time"]
plt.plot( times, tmax_bos-tmin_bos )
plt.xticks(rotation = 90)
plt.xlabel("Date")
plt.ylabel(r"Degrees $^{\circ}$C")

In [ ]:
import hvplot.xarray
# Use groupby processing
ds.air.hvplot(groupby="time", clim=(-5, 40) )

In [ ]:
# Derivatives
ds.air[0,:,:].plot.contour(levels=30,add_colorbar=True)
plt.show()
ds.differentiate("lon").air[0,:,:].plot.contour(levels=30,add_colorbar=True)

In [ ]:
#
# groupby processing can use predefined groups, for example "season" 
#
?ds.groupby
seasonal_mean = ds.groupby("time.season").mean()
seasonal_mean = seasonal_mean.reindex(season=["DJF", "MAM", "JJA", "SON"])
seasonal_mean.air.plot.contour(col="season", levels=20, add_colorbar=True)

In [ ]:
#
#  xarray can be used with netCDF (a widely used self-describing data "container" standard in Earth Science)
#  to look at remote sensed satellite data downloaded from NASA cloud sites.
#
#  Example reading NASA satellite data
#
#

In [ ]:
import s3fs
import datetime as dt
import fsspec
import netCDF4
import requests

In [ ]:
fs = fsspec.filesystem("s3", anon=True)
# OR_ABI-L2-SSTF-M6_G17_s20212840000319_e20212840059385_c20212840104218.nc
# fl=fs.glob("s3://noaa-goes17/ABI-L1b-RadF/2021/284/00/*M6C03*")
fl=fs.glob("s3://noaa-goes17/ABI-L2-SSTF/2021/284/00/*M6_*")

In [ ]:
fn="https://noaa-goes17.s3.amazonaws.com/"+'/'.join(fl[0].split('/')[1:])
resp = requests.get(fn)

In [ ]:
nc4_ds = netCDF4.Dataset("myds",memory=resp.content)
store = xr.backends.NetCDF4DataStore(nc4_ds)
DS = xr.open_dataset(store)

In [ ]:
DS

In [ ]:
#
# xarray includes a masking function that can be used to select data with a particular
# quality flag and in a particular range.
#
masked = DS.SST.where( (DS.DQF==0) & (DS.SST-273.15>5 ) & (DS.SST-273.15<35 ) )

In [ ]:
plt.figure(figsize=(100, 100))
#plt.imshow(masked-273.15,cmap='prism')
#plt.imshow(masked-273.15,cmap='gray')
plt.imshow(masked-273.15,cmap='gist_ncar')
foo=plt.colorbar()
foo.ax.tick_params(axis='y',labelsize=96)
# ax.xaxis.label.set_size(20)

In [ ]:
#
# Now we can look at some climate model SST
#

import xarray as xr
import requests
import netCDF4


fn="https://esgf-world.s3.amazonaws.com/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/historical/r1i1p1f1/Oday/tos/gr/v20180701/tos_Oday_GFDL-CM4_historical_r1i1p1f1_gr_20100101-20141231.nc"
resp = requests.get(fn)

nc4_ds = netCDF4.Dataset("myds",memory=resp.content)
store = xr.backends.NetCDF4DataStore(nc4_ds)
DS_model = xr.open_dataset(store)
DS_model.tos.isel( time=284 ).plot(vmin=5,vmax=35,cmap='gist_ncar')